In [11]:
# Import Required Libraries
import backtrader as bt
import backtrader.feeds as btfeeds
import pandas as pd
import datetime
import numpy as np
import os
import talib

In [12]:
# # Install TA-Lib if not available
# try:
#     import talib
#     print("✓ TA-Lib is available")
# except ImportError:
#     print("Installing TA-Lib...")
#     import subprocess
#     import sys
    
#     # Try to install TA-Lib
#     try:
#         subprocess.check_call([sys.executable, "-m", "pip", "install", "TA-Lib"])
#         import talib
#         print("✓ TA-Lib installed successfully")
#     except Exception as e:
#         print(f"❌ Failed to install TA-Lib: {e}")
#         print("💡 Please install TA-Lib manually:")
#         print("   conda install -c conda-forge ta-lib")
#         print("   or download from: https://www.lfd.uci.edu/~gohlke/pythonlibs/#ta-lib")

In [13]:
long_conditions = {
        'bullish_reversal_patterns': [
            'CDLHAMMER',              # Hammer
            'CDLINVERTEDHAMMER',      # Inverted Hammer
            'CDLMORNINGSTAR',         # Morning Star
            'CDLMORNINGDOJISTAR',     # Morning Doji Star
            'CDLENGULFING',           # Bullish Engulfing
            'CDLPIERCING',            # Piercing Pattern
            'CDLHARAMI',              # Bullish Harami
            'CDLHARAMICROSS',         # Bullish Harami Cross
            'CDLTAKURI',              # Takuri (Dragonfly Doji)
        ],
        
        'bullish_continuation_patterns': [
            'CDL3WHITESOLDIERS',      # Three White Soldiers
            'CDLRISEFALL3METHODS',    # Rising Three Methods
            'CDLMATHOLD',             # Mat Hold
            'CDLSEPARATINGLINES',     # Bullish Separating Lines
            'CDLTASUKIGAP',           # Bullish Tasuki Gap uptrend
        ],
        
        'bullish_bottom_patterns': [
            'CDLABANDONEDBABY',       # Abandoned Baby
            'CDLLADDERBOTTOM',        # Ladder Bottom
            'CDLMATCHINGLOW',         # Matching Low
            'CDLUNIQUE3RIVER',        # Unique Three River
        ],
        
        'bullish_special_patterns': [
            'CDL3INSIDE',             # Three Inside Up
            'CDL3OUTSIDE',            # Three Outside Up
            'CDLBELTHOLD',            # Belt Hold
            'CDLBREAKAWAY',           # Breakaway
            'CDLKICKING',             # Kicking
            'CDLKICKINGBYLENGTH',     # Kicking By Length
            'CDLSTICKSANDWICH',       # Stick Sandwich
        ]
    }

In [14]:
# Strategy Candlestick Patterns - FIXED VERSION with Proper Risk Management

class Long_Candlesticks(bt.Strategy):
    """
    Strategy focusing on long candlestick patterns for trade entries
    Entry signals based on bullish patterns using TA-Lib candlestick functions
    
    DESIGNED FOR HOURLY DATA (1H timeframe)
    Stop-loss and take-profit based on ATR
    
    Expected data: S&P 500 hourly OHLCV bars
    Timeframe: 1H (hourly candles)
    """
    
    params = (
        ('printlog', True),  # Enable logging to debug issues
        ('atr_period', 14),
        ('atr_sl_multiplier', 2.0),
        ('atr_tp_multiplier', 3.0),
        ('position_pct', 0.05),  # Use only 5% of portfolio per trade for safety
        ('max_trades', 10),  # Limit total number of trades
        ('lookback_period', 50),  # Number of bars to look back for pattern detection
    )

    def __init__(self):
        # ATR for stop-loss and take-profit calculations
        self.atr = bt.indicators.ATR(self.data, period=self.p.atr_period)

        self.stop_loss = None
        self.take_profit = None
        
        self.order = None
        self.trade_count = 0
        
        # Store OHLC data for TA-Lib calculations
        self.open_data = []
        self.high_data = []
        self.low_data = []
        self.close_data = []

    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price:.2f}')
            else:
                self.log(f'SELL EXECUTED, Price: {order.executed.price:.2f}')
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.trade_count += 1
        self.log(f'TRADE #{self.trade_count}: PROFIT {trade.pnl:.2f}', doprint=True)

    def check_bullish_patterns(self, open_prices, high_prices, low_prices, close_prices):
        """Check for bullish candlestick patterns using TA-Lib"""
        bullish_signals = []
        
        try:
            # Convert to numpy arrays
            open_arr = np.array(open_prices, dtype=np.float64)
            high_arr = np.array(high_prices, dtype=np.float64)
            low_arr = np.array(low_prices, dtype=np.float64)
            close_arr = np.array(close_prices, dtype=np.float64)
            
            # Check bullish reversal patterns
            hammer = talib.CDLHAMMER(open_arr, high_arr, low_arr, close_arr)
            inverted_hammer = talib.CDLINVERTEDHAMMER(open_arr, high_arr, low_arr, close_arr)
            morning_star = talib.CDLMORNINGSTAR(open_arr, high_arr, low_arr, close_arr)
            morning_doji = talib.CDLMORNINGDOJISTAR(open_arr, high_arr, low_arr, close_arr)
            engulfing = talib.CDLENGULFING(open_arr, high_arr, low_arr, close_arr)
            piercing = talib.CDLPIERCING(open_arr, high_arr, low_arr, close_arr)
            harami = talib.CDLHARAMI(open_arr, high_arr, low_arr, close_arr)

            # added patterns
            takuri = talib.CDLTAKURI(open_arr, high_arr, low_arr, close_arr)  # Takuri (Dragonfly Doji)
            rise_fall = talib.CDLRISEFALL3METHODS(open_arr, high_arr, low_arr, close_arr)  # Rising Three Methods
            mat_hold = talib.CDLMATHOLD(open_arr, high_arr, low_arr, close_arr)  # Mat Hold
            separating_lines = talib.CDLSEPARATINGLINES(open_arr, high_arr, low_arr, close_arr)  # Bullish Separating Lines
            tasuki_gap = talib.CDLTASUKIGAP(open_arr, high_arr, low_arr, close_arr)  # Bullish Tasuki Gap uptrend
            abandoned_baby = talib.CDLABANDONEDBABY(open_arr, high_arr, low_arr, close_arr)  # Abandoned Baby
            ladder_bottom = talib.CDLLADDERBOTTOM(open_arr, high_arr, low_arr, close_arr)  # Ladder Bottom
            matching_low = talib.CDLMATCHINGLOW(open_arr, high_arr, low_arr, close_arr)  # Matching Low
            unique_three_river = talib.CDLUNIQUE3RIVER(open_arr, high_arr, low_arr, close_arr)  # Unique Three River
            three_inside = talib.CDL3INSIDE(open_arr, high_arr, low_arr, close_arr)  # Three Inside Up
            three_outside = talib.CDL3OUTSIDE(open_arr, high_arr, low_arr, close_arr)  # Three Outside Up
            belt_hold = talib.CDLBELTHOLD(open_arr, high_arr, low_arr, close_arr)  # Belt Hold
            breakaway = talib.CDLBREAKAWAY(open_arr, high_arr, low_arr, close_arr)  # Breakaway
            kicking = talib.CDLKICKING(open_arr, high_arr, low_arr, close_arr)  # Kicking
            kicking_by_length = talib.CDLKICKINGBYLENGTH(open_arr, high_arr, low_arr, close_arr)  # Kicking By Length
            sticks_sandwich = talib.CDLSTICKSANDWICH(open_arr, high_arr, low_arr, close_arr)  # Stick Sandwich

            # Check bullish continuation patterns
            three_white_soldiers = talib.CDL3WHITESOLDIERS(open_arr, high_arr, low_arr, close_arr)
            
            # Combine all bullish signals (positive values indicate bullish patterns)
            patterns = [hammer, inverted_hammer, morning_star, morning_doji, 
                       engulfing, piercing, harami, three_white_soldiers, 
                       takuri, rise_fall, mat_hold, separating_lines, tasuki_gap, 
                       abandoned_baby, ladder_bottom, matching_low, unique_three_river, 
                       three_inside, three_outside, belt_hold, breakaway, kicking, 
                       kicking_by_length, sticks_sandwich]

            # Check if any pattern shows a bullish signal (> 0) in the last bar
            for pattern in patterns:
                if len(pattern) > 0 and pattern[-1] > 0:
                    bullish_signals.append(pattern[-1])
            
            return len(bullish_signals) > 0
            
        except Exception as e:
            self.log(f"Pattern detection error: {e}")
            return False

    def next(self):
        # Store current OHLC data
        self.open_data.append(float(self.data.open[0]))
        self.high_data.append(float(self.data.high[0]))
        self.low_data.append(float(self.data.low[0]))
        self.close_data.append(float(self.data.close[0]))
        
        # Keep only the last lookback_period bars
        if len(self.close_data) > self.p.lookback_period:
            self.open_data = self.open_data[-self.p.lookback_period:]
            self.high_data = self.high_data[-self.p.lookback_period:]
            self.low_data = self.low_data[-self.p.lookback_period:]
            self.close_data = self.close_data[-self.p.lookback_period:]
        # Wait for enough data for pattern detection and ATR to stabilize
        if len(self.close_data) < 30 or len(self.data) < 60:
            return
            
        # Ensure ATR has valid values
        if not self.atr or len(self.atr) == 0 or self.atr[0] is None or np.isnan(self.atr[0]):
            return

        if self.order:
            return

        if not self.position:
            # Check for bullish candlestick patterns
            if self.check_bullish_patterns(self.open_data, self.high_data, 
                                         self.low_data, self.close_data):
                self.log('BUY SIGNAL: Bullish candlestick pattern detected')
                
                # SAFE position sizing calculation
                if self.trade_count >= self.p.max_trades:
                    self.log(f'Max trades ({self.p.max_trades}) reached, skipping entry')
                    return
                
                cash = self.broker.getcash()
                portfolio_value = self.broker.getvalue()
                price = self.data.close[0]
                
                # Calculate position size with multiple safety checks
                position_value = portfolio_value * self.p.position_pct
                max_position_value = min(position_value, cash * 0.95)  # Never use more than 95% of cash
                size = max(1, int(max_position_value / price))  # Ensure at least 1 share
                cost = size * price
                
                self.log(f'POSITION CALC: Portfolio=${portfolio_value:.2f}, Cash=${cash:.2f}, Price=${price:.2f}')
                self.log(f'Size={size}, Cost=${cost:.2f}, Position%={self.p.position_pct*100:.1f}%')
                
                # Safety check: ensure we can afford the position
                if cost <= cash and size > 0 and cost < portfolio_value * 0.2:  # Never risk more than 20%
                    self.order = self.buy(size=size)
                    # Set stop-loss and take-profit based on ATR
                    self.stop_loss = price - (self.atr[0] * self.p.atr_sl_multiplier)
                    self.take_profit = price + (self.atr[0] * self.p.atr_tp_multiplier)
                    self.log(f'BUY ORDER: Size={size}, Price=${price:.2f}, Cost=${cost:.2f}')
                    self.log(f'SL=${self.stop_loss:.2f}, TP=${self.take_profit:.2f}, ATR=${self.atr[0]:.2f}')
                else:
                    self.log(f'POSITION REJECTED: Cost=${cost:.2f} > Available=${cash:.2f} or unsafe size')
        else:
            # Manage existing position with ATR-based stop-loss and take-profit
            if self.stop_loss is not None and self.data.close[0] <= self.stop_loss:
                self.log('STOP-LOSS HIT')
                self.order = self.sell()
            elif self.take_profit is not None and self.data.close[0] >= self.take_profit:
                self.log('TAKE-PROFIT HIT')
                self.order = self.sell()

    def stop(self):
        self.log(f'Long_Candlesticks Strategy: {self.trade_count} trades, Final Value: ${self.broker.getvalue():.2f}', doprint=True)

print("✓ Long_Candlesticks Strategy created - using TA-Lib candlestick patterns!")

✓ Long_Candlesticks Strategy created - using TA-Lib candlestick patterns!


In [15]:
# CSV Files List for Multi-Year Quarterly Backtesting
import glob
import os

# List of quarterly CSV files for backtesting
csv_files = [
    'USA500IDXUSD_H1_returns_2019_q4.csv',
    'USA500IDXUSD_H1_returns_2020_q4.csv', 
    'USA500IDXUSD_H1_returns_2021_q4.csv',
    'USA500IDXUSD_H1_returns_2022_q4.csv',
    'USA500IDXUSD_H1_returns_2023_q4.csv',
    'USA500IDXUSD_H1_returns_2024_q4.csv'
]

# Automatically detect all Q4 CSV files in the data directory
data_dir = '01_data'
auto_detected_files = glob.glob(os.path.join(data_dir, '*_q4.csv'))

print(f"📂 Manual CSV list: {len(csv_files)} files")
print(f"🔍 Auto-detected files: {len(auto_detected_files)} files")

# Use auto-detected files if available, otherwise use manual list
if auto_detected_files:
    csv_files = [os.path.basename(f) for f in auto_detected_files]
    csv_files.sort()  # Sort chronologically
    print("✅ Using auto-detected files:")
else:
    print("✅ Using manual file list:")

for i, file in enumerate(csv_files, 1):
    full_path = os.path.join(data_dir, file)
    exists = "✓" if os.path.exists(full_path) else "❌"
    print(f"  {i}. {exists} {file}")

print(f"\n🎯 Ready to backtest {len(csv_files)} quarterly datasets")

📂 Manual CSV list: 6 files
🔍 Auto-detected files: 6 files
✅ Using auto-detected files:
  1. ✓ USA500IDXUSD_H1_returns_2019_q4.csv
  2. ✓ USA500IDXUSD_H1_returns_2020_q4.csv
  3. ✓ USA500IDXUSD_H1_returns_2021_q4.csv
  4. ✓ USA500IDXUSD_H1_returns_2022_q4.csv
  5. ✓ USA500IDXUSD_H1_returns_2023_q4.csv
  6. ✓ USA500IDXUSD_H1_returns_2024_q4.csv

🎯 Ready to backtest 6 quarterly datasets


In [16]:
# Backtest Function for Long_Candlesticks Strategy

def test_long_candlesticks_strategy(csv_filename=None):
    """Test the Long_Candlesticks strategy with S&P 500 quarterly data"""
    
    # Use provided filename or default to 2023 Q4
    if csv_filename is None:
        csv_filename = 'USA500IDXUSD_H1_returns_2023_q4.csv'
    
    print(f"=== TESTING Long_Candlesticks on {csv_filename} ===\n")
    
    cerebro = bt.Cerebro()
    
    # Load S&P 500 data
    spy_file = os.path.join('01_data', csv_filename)
    df = pd.read_csv(spy_file)
    
    # Combine date and timestamp columns
    df['DateTime'] = pd.to_datetime(df['date'] + ' ' + df['timestamp'])
    
    # Rename columns to match expected format
    df = df.rename(columns={
        'open': 'Open',
        'high': 'High', 
        'low': 'Low',
        'close': 'Close',
        'volume': 'Volume'
    })
    
    # Select only needed columns and use DateTime
    df = df[['DateTime', 'Open', 'High', 'Low', 'Close', 'Volume']].copy()
    df = df.rename(columns={'DateTime': 'Date'})
    df = df.sort_values('Date').dropna()
    
    # Use full quarterly dataset (Q4 data)
    start_date = df['Date'].min()
    end_date = df['Date'].max()
    df_recent = df.copy()
    
    print(f"📊 Data: {len(df_recent)} hourly bars from {start_date.date()} to {end_date.date()}")
    print(f"📅 Dataset: {csv_filename} (Full Q4 period)")
    
    # Create temporary file for backtrader
    temp_file = 'temp_simplified_test.csv'
    df_recent.to_csv(temp_file, index=False)
    
    # Create data feed
    data = btfeeds.GenericCSVData(
        dataname=temp_file,
        dtformat=('%Y-%m-%d %H:%M:%S'),
        datetime=0, open=1, high=2, low=3, close=4, volume=5,
        openinterest=-1, headers=True,
    )

    # Add data and strategy
    cerebro.adddata(data)
    cerebro.addstrategy(Long_Candlesticks, printlog=False)
    
    # Set initial capital and commission
    initial_capital = 50000.0
    cerebro.broker.setcash(initial_capital)
    cerebro.broker.setcommission(commission=0.001)  # 0.1% commission
    cerebro.broker.set_slippage_perc(0.01)  # 1% slippage for realism
    
    # Add analyzers
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trades")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")

    print(f"💰 Starting Capital: ${initial_capital:,.2f}")
    print("🚀 Running backtest...\n")
    
    # Run backtest
    results = cerebro.run()
    
    # Clean up temporary file
    if os.path.exists(temp_file):
        os.remove(temp_file)
    
    # Calculate results
    final_value = cerebro.broker.getvalue()
    total_return = ((final_value - initial_capital) / initial_capital) * 100
    
    print(f"\n=== BACKTEST RESULTS ===")
    print(f"💰 Final Portfolio Value: ${final_value:,.2f}")
    print(f"📈 Total Return: {total_return:.2f}%")
    print(f"💵 Net Profit: ${final_value - initial_capital:,.2f}")
    
    # Analyze trades
    strat = results[0]
    trade_analysis = strat.analyzers.trades.get_analysis()
    
    if trade_analysis and hasattr(trade_analysis, 'total'):
        total_trades = getattr(trade_analysis.total, 'total', 0)
        print(f"\n=== TRADE ANALYSIS ===")
        print(f"📊 Total Trades: {total_trades}")
        
        # Winning trades
        if hasattr(trade_analysis, 'won'):
            won_trades = getattr(trade_analysis.won, 'total', 0)
            won_pnl = getattr(trade_analysis.won, 'pnl', {}).get('total', 0)
            win_rate = (won_trades / total_trades * 100) if total_trades > 0 else 0
            print(f"✅ Winning Trades: {won_trades} ({win_rate:.1f}%)")
            if won_pnl:
                print(f"💰 Winning PnL: ${won_pnl:.2f}")
        
        # Losing trades  
        if hasattr(trade_analysis, 'lost'):
            lost_trades = getattr(trade_analysis.lost, 'total', 0)
            lost_pnl = getattr(trade_analysis.lost, 'pnl', {}).get('total', 0)
            print(f"❌ Losing Trades: {lost_trades}")
            if lost_pnl:
                print(f"💸 Losing PnL: ${lost_pnl:.2f}")
    
    # Performance metrics
    print(f"\n=== PERFORMANCE METRICS ===")
    
    # Sharpe Ratio
    sharpe_analysis = strat.analyzers.sharpe.get_analysis()
    if sharpe_analysis and 'sharperatio' in sharpe_analysis:
        sharpe_ratio = sharpe_analysis['sharperatio'] or 0
        print(f"📊 Sharpe Ratio: {sharpe_ratio:.4f}")

    # Drawdown Analysis
    drawdown_analysis = strat.analyzers.drawdown.get_analysis()
    if drawdown_analysis:
        max_dd = drawdown_analysis.get('max', {}).get('drawdown', 0)
        print(f"📉 Max Drawdown: {max_dd:.2f}%")
    
    return results, total_trades, final_value

print("✓ Backtest function ready for Long_Candlesticks")

✓ Backtest function ready for Long_Candlesticks


In [17]:
# Multi-File Backtesting Function

def run_multi_file_backtests(csv_files_list):
    """Run backtests on multiple CSV files and collect comprehensive results"""
    
    print("🚀 STARTING MULTI-FILE BACKTESTING CAMPAIGN")
    print("=" * 60)
    
    all_results = {}
    summary_data = []
    
    for i, csv_file in enumerate(csv_files_list, 1):
        print(f"\n📊 BACKTEST {i}/{len(csv_files_list)}: {csv_file}")
        print("-" * 50)
        
        try:
            # Check if file exists
            full_path = os.path.join('01_data', csv_file)
            if not os.path.exists(full_path):
                print(f"❌ File not found: {csv_file}")
                continue
            
            # Run individual backtest
            results, trade_count, final_value = test_long_candlesticks_strategy(csv_file)
            
            # Extract detailed analytics
            strat = results[0]
            trade_analysis = strat.analyzers.trades.get_analysis()
            sharpe_analysis = strat.analyzers.sharpe.get_analysis()
            drawdown_analysis = strat.analyzers.drawdown.get_analysis()
            
            # Calculate performance metrics
            initial_capital = 50000.0
            total_return = ((final_value - initial_capital) / initial_capital) * 100
            net_profit = final_value - initial_capital
            
            # Extract trade statistics
            total_trades = getattr(trade_analysis.total, 'total', 0) if hasattr(trade_analysis, 'total') else 0
            won_trades = getattr(trade_analysis.won, 'total', 0) if hasattr(trade_analysis, 'won') else 0
            lost_trades = getattr(trade_analysis.lost, 'total', 0) if hasattr(trade_analysis, 'lost') else 0
            win_rate = (won_trades / total_trades * 100) if total_trades > 0 else 0
            
            won_pnl = getattr(trade_analysis.won, 'pnl', {}).get('total', 0) if hasattr(trade_analysis, 'won') else 0
            lost_pnl = getattr(trade_analysis.lost, 'pnl', {}).get('total', 0) if hasattr(trade_analysis, 'lost') else 0
            
            sharpe_ratio = sharpe_analysis.get('sharperatio', 0) if sharpe_analysis else 0
            max_drawdown = drawdown_analysis.get('max', {}).get('drawdown', 0) if drawdown_analysis else 0
            
            # Store results
            file_result = {
                'csv_file': csv_file,
                'final_value': final_value,
                'total_return_pct': total_return,
                'net_profit': net_profit,
                'total_trades': total_trades,
                'won_trades': won_trades,
                'lost_trades': lost_trades,
                'win_rate_pct': win_rate,
                'won_pnl': won_pnl,
                'lost_pnl': lost_pnl,
                'sharpe_ratio': sharpe_ratio,
                'max_drawdown_pct': max_drawdown
            }
            
            all_results[csv_file] = file_result
            summary_data.append(file_result)
            
            # Print results for this file
            print(f"\n📈 DETAILED RESULTS FOR {csv_file}:")
            print(f"💰 Final Value: ${final_value:,.2f}")
            print(f"📊 Total Return: {total_return:.2f}%")
            print(f"💵 Net Profit: ${net_profit:,.2f}")
            print(f"📈 Total Trades: {total_trades}")
            print(f"✅ Won: {won_trades} ({win_rate:.1f}%) | PnL: ${won_pnl:.2f}")
            print(f"❌ Lost: {lost_trades} | PnL: ${lost_pnl:.2f}")
            print(f"📊 Sharpe Ratio: {sharpe_ratio:.4f}")
            print(f"📉 Max Drawdown: {max_drawdown:.2f}%")
            
        except Exception as e:
            print(f"❌ ERROR in {csv_file}: {str(e)}")
            continue
    
    # Print comprehensive summary
    print(f"\n\n🎯 COMPREHENSIVE BACKTESTING SUMMARY")
    print("=" * 70)
    
    if summary_data:
        total_files = len(summary_data)
        avg_return = sum(d['total_return_pct'] for d in summary_data) / total_files
        avg_trades = sum(d['total_trades'] for d in summary_data) / total_files
        avg_win_rate = sum(d['win_rate_pct'] for d in summary_data) / total_files
        avg_sharpe = sum(d['sharpe_ratio'] for d in summary_data) / total_files
        avg_drawdown = sum(d['max_drawdown_pct'] for d in summary_data) / total_files
        
        best_performer = max(summary_data, key=lambda x: x['total_return_pct'])
        worst_performer = min(summary_data, key=lambda x: x['total_return_pct'])
        
        print(f"📊 Backtested Files: {total_files}")
        print(f"📈 Average Return: {avg_return:.2f}%")
        print(f"🔢 Average Trades per Quarter: {avg_trades:.1f}")
        print(f"🎯 Average Win Rate: {avg_win_rate:.1f}%")
        print(f"📊 Average Sharpe Ratio: {avg_sharpe:.4f}")
        print(f"📉 Average Max Drawdown: {avg_drawdown:.2f}%")
        
        print(f"\n🏆 BEST PERFORMER:")
        print(f"   📁 {best_performer['csv_file']}")
        print(f"   📈 Return: {best_performer['total_return_pct']:.2f}%")
        
        print(f"\n📉 WORST PERFORMER:")
        print(f"   📁 {worst_performer['csv_file']}")
        print(f"   📈 Return: {worst_performer['total_return_pct']:.2f}%")
        
        print(f"\n📋 DETAILED PERFORMANCE TABLE:")
        print("-" * 100)
        print(f"{'File':<35} {'Return %':<10} {'Trades':<8} {'Win %':<8} {'Sharpe':<10} {'DD %':<8}")
        print("-" * 100)
        
        for data in summary_data:
            print(f"{data['csv_file']:<35} {data['total_return_pct']:>8.2f}% {data['total_trades']:>6.0f}   {data['win_rate_pct']:>6.1f}%  {data['sharpe_ratio']:>8.4f}  {data['max_drawdown_pct']:>6.2f}%")
    
    else:
        print("❌ No successful backtests completed")
    
    return all_results

print("✅ Multi-file backtesting function ready!")

✅ Multi-file backtesting function ready!


In [18]:
# Execute Long_Candlesticks Backtest
try:
    results, trade_count, final_value = test_long_candlesticks_strategy()
    
    print(f"\n🎉 BACKTEST COMPLETED SUCCESSFULLY!")
    print(f"📊 Summary: {trade_count} trades, Final Value: ${final_value:,.2f}")
    
    if trade_count > 10:
        print("✅ Strategy shows active trading with multiple trades")
    elif trade_count > 1:
        print("📈 Strategy executed multiple trades")
    else:
        print("⚠️ Strategy was very selective with few trades")
        
except Exception as e:
    print(f"❌ Error running backtest: {str(e)}")
    print("💡 Make sure the S&P 500 data file exists at the specified path")

=== TESTING Long_Candlesticks on USA500IDXUSD_H1_returns_2023_q4.csv ===

📊 Data: 1463 hourly bars from 2023-10-02 to 2023-12-29
📅 Dataset: USA500IDXUSD_H1_returns_2023_q4.csv (Full Q4 period)
💰 Starting Capital: $50,000.00
🚀 Running backtest...

2023-10-09, TRADE #1: PROFIT 38.23
2023-12-29, Long_Candlesticks Strategy: 1 trades, Final Value: $-289181.38

=== BACKTEST RESULTS ===
💰 Final Portfolio Value: $-289,181.38
📈 Total Return: -678.36%
💵 Net Profit: $-339,181.38

=== TRADE ANALYSIS ===
📊 Total Trades: 2
✅ Winning Trades: 1 (50.0%)
💰 Winning PnL: $12.65
❌ Losing Trades: 0

=== PERFORMANCE METRICS ===
📊 Sharpe Ratio: 0.0000
📉 Max Drawdown: 430.42%

🎉 BACKTEST COMPLETED SUCCESSFULLY!
📊 Summary: 2 trades, Final Value: $-289,181.38
📈 Strategy executed multiple trades


In [ ]:
# # Execute Multi-File Backtesting Campaign

# # Run backtests on all quarterly CSV files
# print("🎯 LAUNCHING COMPREHENSIVE QUARTERLY BACKTESTING")
# print("🔄 This will run the Long_Candlesticks strategy on each Q4 dataset")
# print("⏰ Please wait while processing multiple datasets...\n")

# try:
#     # Execute the multi-file backtesting
#     all_backtest_results = run_multi_file_backtests(csv_files)
    
#     print(f"\n🎉 MULTI-FILE BACKTESTING COMPLETED!")
#     print(f"📊 Successfully processed {len(all_backtest_results)} datasets")
    
#     # Additional insights
#     if all_backtest_results:
#         profitable_quarters = sum(1 for result in all_backtest_results.values() 
#                                  if result['total_return_pct'] > 0)
#         total_quarters = len(all_backtest_results)
#         profitability_rate = (profitable_quarters / total_quarters * 100) if total_quarters > 0 else 0
        
#         print(f"💰 Profitable Quarters: {profitable_quarters}/{total_quarters} ({profitability_rate:.1f}%)")
        
#         # Strategy consistency analysis
#         returns = [result['total_return_pct'] for result in all_backtest_results.values()]
#         avg_return = sum(returns) / len(returns) if returns else 0
#         return_std = np.std(returns) if returns else 0
        
#         print(f"📈 Return Statistics:")
#         print(f"   • Average: {avg_return:.2f}%")
#         print(f"   • Std Dev: {return_std:.2f}%")
#         print(f"   • Range: {min(returns):.2f}% to {max(returns):.2f}%")
        
#         # Trading activity analysis
#         trade_counts = [result['total_trades'] for result in all_backtest_results.values()]
#         avg_trades = sum(trade_counts) / len(trade_counts) if trade_counts else 0
        
#         print(f"🔢 Trading Activity:")
#         print(f"   • Average trades per quarter: {avg_trades:.1f}")
#         print(f"   • Total trades across all quarters: {sum(trade_counts)}")
        
#         print(f"\n✅ Long_Candlesticks strategy analysis complete across {len(csv_files)} quarterly periods")
    
# except Exception as e:
#     print(f"❌ Error in multi-file backtesting: {str(e)}")
#     print("💡 Check that all CSV files exist in the 01_data directory")
#     print("🔍 You can also run individual backtests by calling:")
#     print("   test_long_candlesticks_strategy('specific_file.csv')")

🎯 LAUNCHING COMPREHENSIVE QUARTERLY BACKTESTING
🔄 This will run the Long_Candlesticks strategy on each Q4 dataset
⏰ Please wait while processing multiple datasets...

🚀 STARTING MULTI-FILE BACKTESTING CAMPAIGN

📊 BACKTEST 1/6: USA500IDXUSD_H1_returns_2019_q4.csv
--------------------------------------------------
=== TESTING Long_Candlesticks on USA500IDXUSD_H1_returns_2019_q4.csv ===

📊 Data: 1485 hourly bars from 2019-10-01 to 2019-12-31
📅 Dataset: USA500IDXUSD_H1_returns_2019_q4.csv (Full Q4 period)
💰 Starting Capital: $50,000.00
🚀 Running backtest...

2019-10-07, TRADE #1: PROFIT 36.80
2019-12-31, Long_Candlesticks Strategy: 1 trades, Final Value: $-134462.92

=== BACKTEST RESULTS ===
💰 Final Portfolio Value: $-134,462.92
📈 Total Return: -368.93%
💵 Net Profit: $-184,462.92

=== TRADE ANALYSIS ===
📊 Total Trades: 2
✅ Winning Trades: 1 (50.0%)
💰 Winning PnL: $25.11
❌ Losing Trades: 0

=== PERFORMANCE METRICS ===
📊 Sharpe Ratio: 0.0000
📉 Max Drawdown: 403.76%

📈 DETAILED RESULTS FOR US

In [ ]:
# # Validate Hourly Data Structure
# import pandas as pd
# import os

# # Quick data validation
# spy_file = os.path.join('01_data', 'USA500IDXUSD_H1_returns_2019-2024_q4.csv')
# df = pd.read_csv(spy_file)

# print("=== HOURLY DATA VALIDATION ===")
# print(f"📊 Total records: {len(df):,}")
# print(f"📅 Date range: {df.date.min()} to {df.date.max()}")

# # Check time intervals
# df['DateTime'] = pd.to_datetime(df['date'] + ' ' + df['timestamp'])
# df_sorted = df.sort_values('DateTime')

# # Calculate time differences between consecutive bars
# time_diffs = df_sorted['DateTime'].diff().dropna()
# most_common_diff = time_diffs.mode().iloc[0] if len(time_diffs) > 0 else None

# print(f"🕐 Most common time interval: {most_common_diff}")
# print(f"📈 Price range: ${df.close.min():.2f} - ${df.close.max():.2f}")

# # Show sample of hourly progression
# print(f"\\n🔍 Sample hourly data progression:")
# sample = df_sorted[['date', 'timestamp', 'close']].head(10)
# for idx, row in sample.iterrows():
#     print(f"  {row['date']} {row['timestamp']} - Close: ${row['close']:.2f}")

# # Verify we have proper hourly intervals
# hourly_count = (time_diffs == pd.Timedelta(hours=1)).sum()
# total_intervals = len(time_diffs)
# hourly_percentage = (hourly_count / total_intervals * 100) if total_intervals > 0 else 0

# print(f"\\n✅ Hourly intervals: {hourly_count:,} / {total_intervals:,} ({hourly_percentage:.1f}%)")

# if hourly_percentage > 90:
#     print("✅ Data confirmed as hourly timeframe")
# elif hourly_percentage > 50:
#     print("⚠️ Mixed timeframe data detected") 
# else:
#     print("❌ Data does not appear to be hourly")

FileNotFoundError: [Errno 2] No such file or directory: '01_data\\USA500IDXUSD_H1_returns_2019-2024_q4.csv'